In [2]:
import pandas as pd
import numpy as np
import urllib.error as err
import urllib.request as libreq
import urllib
import time, socket, sys, os
from bs4 import BeautifulSoup as bs

## READ IN CSV

In [44]:
player = []
team = []

for dire in os.listdir(os.getcwd()):
    if "phs_" in dire:
        cur = os.path.join(os.getcwd(), dire, "")
        for file in os.listdir(cur):
            try:
                df = pd.read_csv(cur + file)[['player', 'team']].to_numpy()
                player += set(df[:, 0].tolist())
                team   += set(df[:, 1].tolist())
            except:
                df = pd.read_csv(cur + file)[['player_name', 'team_name']].to_numpy()
                player += set(df[:, 0].tolist())
                team   += set(df[:, 1].tolist())
                
player = list(set(player))
team   = list(set(team))
player = sorted(player)
team   = sorted(team)
#player, team

## Clean up player name in url

In [45]:
for pl in range(len(player)):
    if player[pl].lower() == 'although':
        player[pl] = 'ALTHOUGH'
    elif player[pl].lower() == 'bebe':
        player[pl] = 'BeBe'
    elif player[pl].lower() == 'bumper':
        player[pl] = 'Bumper'
    elif player[pl].lower() == 'choihyobin':
        player[pl] = 'ChoiHyoBin'
    elif player[pl].lower() == 'colourhex':
        player[pl] = 'Colourhex'
    elif player[pl].lower() == 'dreamer':
        player[pl] = 'Dreamer'
    elif player[pl].lower() == 'fdgod':
        player[pl] = 'FDGoD'
    elif player[pl].lower() == 'fearless':
        player[pl] = 'Fearless'
    elif player[pl].lower() == 'fits':
        player[pl] = 'FITS'
    elif player[pl].lower() == 'gargoyle':
        player[pl] = 'Gargoyle'
    elif player[pl].lower() == 'garry':
        player[pl] = 'Garry'
    elif player[pl].lower() == 'gangnamjin':
        player[pl] = 'Gangnamjin'
    elif player[pl].lower() == 'izayaki':
        player[pl] = 'Izayaki'
    elif player[pl].lower() == 'jiqiren':
        player[pl] = 'jiqiren'
    elif player[pl].lower() == 'jake':
        player[pl] = 'JAKE'
    elif player[pl].lower() == 'jmac':
        player[pl] = 'JMAC'   
    elif player[pl].lower() == 'leejaegon':
        player[pl] = 'LeeJaeGon'
    elif player[pl].lower() == 'mcgravy':
        player[pl] = 'McGravy'
    elif player[pl].lower() == 'onlywish':
        player[pl] = 'OnlyWish'
    elif player[pl].lower() == 'quatermain':
        player[pl] = 'Quatermain'
    elif player[pl].lower() == 'saebyeolbe':
        player[pl] = 'Saebyeolbe'
    elif player[pl].lower() == 'sp9rk1e':
        player[pl] = 'Sp9rk1e'
    elif player[pl].lower() == 'swon':
        player[pl] = 'Swon'
    elif player[pl].lower() == 'spree':
        player[pl] = 'SPREE'
    elif player[pl].lower() == 'toyou':
        player[pl] = 'Toyou'
    elif player[pl].lower() == 'ttuba':
        player[pl] = 'TTuba'
    elif player[pl].lower() == 'whoru':
        player[pl] = 'WhoRU'
    elif player[pl].lower() == 'zunba':
        player[pl] = 'zunba'
    elif player[pl].lower() == 'blasé' or player[pl].lower() == 'blase':
        player[pl] = 'Blasé'
    elif player[pl].lower() == 'bqb':
        player[pl] = 'BQB'
    elif player[pl].lower() == 'snow':
        player[pl] = 'Snow_(Brazilian_player)'
        player.append('Snow_(Mikias_Yohannes)');

## "global" var before running the acutal soup part

In [38]:
base_url = 'https://liquipedia.net/overwatch/'
list_player = []
already_dealt = []
error_list = []
history_list = []
cols = ['Romanized Name',
  'Birth','Country','Status','Team','Approx. Total Earnings','Common Team Role(s)','Signature Hero','Player Name']

## Actual web crawling

In [40]:
for p in player:
    if p.lower() in already_dealt:
        continue
    print(p)
    
    url = base_url + p

    attempts, success = 0, 0
    while attempts < 2 and not success:
        try:
            with libreq.urlopen(url, timeout=30) as page:  # request info
                doc = page.read().decode('utf-8')
            success = 1
        except (err.URLError, err.HTTPError) as e:  # catch Internet error
            attempts += 1
            print(f"\n{e.reason}-{p}\n")
            error_list.append(p)
            if e.reason == 'NOT FOUND':  # return e if 404: page not found
                print("page not found.")
            time.sleep(1)
        except socket.timeout:  # catch timeout error
            attempts += 1
            print("\nTimeout error\n")
        except UnicodeEncodeError as e:
            with libreq.urlopen("http:" + urllib.parse.quote(url.split(':')[1])) as page:
                doc = page.read()
            success = 1
            print("Not Recognized Characters.")
        except:  # catch all rest error
            attempts += 1
            print("\nUnknown error\n")
        
    if not success:
        continue


    soup = bs(doc)
    t = soup.findAll("div", {"class":'infobox-cell-2'})
    a = soup.findAll("div", {"class":'infobox-center'})
    mylist = []
    for i in range(len(a)):
        te = a[i].text.replace("–", "-").replace("—", "-").replace('\xa0', ' ')  
        if '-' in te:
            mylist.append(te)
            

    t2 = ''.join(mylist).strip().split('\n')
    l = {}
    

    for idx in range(0, len(t), 2):
        s = t[idx].text
        try:
            if s[:-1] not in cols:
                continue
            elif s == 'Birth:':
                l[s[:-1]] = t[idx+1].text.split('(')[1].replace(')', '')
            elif s == 'Signature Hero:' or s == 'Common Team Role(s):':
                l[s[:-1]] = t[idx+1].text.replace('\n', '').replace(':', '').strip().replace(" ", ", ")
            else:
                l[s[:-1]] = t[idx+1].text
        except:
            l[s[:-1]] = t[idx+1].text
            
        #deal with history part

    for idx in range(0, len(t2), 1):
        try:
            l2 = {}
            t2[idx] = t2[idx].replace("–", "-")
            l2['Player Name'] = p
            l2["start_time"] = t2[idx].split(' - ')[0]
            l2["end_time"] = t2[idx].split(' - ')[1].split('  ')[0]
            l2["Team"] = t2[idx].split(' - ')[1].split('  ')[1]
            history_list.append(l2)
        except:
            print("ha"*20)
            
        

    l['Player Name'] = p 
    list_player.append(l)
    already_dealt.append(p.lower())


ANS
ATing
Ado
Adora
Agilities
Aid
AimGod
Alarm
Altering
ALTHOUGH
Anamo
Apply
ArHaN
ArK
Architect
Asher
BeBe
BERNAR
Bumper
Babel
Baconjack
Bani
Bazzi
Bdosin
Beast
BenBest
BiaNcA
BigG00se
Birdring
Bischu
Boink
Boombox
Bunny
CWoosH
Carcar
Chara
ChipSa
Chipshajen
ChoiHyoBin
Clestyn
Clockwork
Closer
CoMa
ColdesT
Colourhex
Corey
Cr0ng
Creative
Crimzo
Custa
DDing
DPI
Dreamer
Daco
Daemin
Dalton
Danteh
DayFly
Decay
Diya
Dogman
Doha
DreamKazper
EFFECT
ELLIVOTE
Edison
Eileen
Elk
Elsa
Envy
Erster
FCTFCTN
FDGoD
Fearless
Fahzix
Fate
Fielder
Finnsi
Fire
Fissure
FITS
Fiveking
Fl0w3R
Fleta
Freefeel
FunnyAstro
Fury
Fusions
Fuze
Gargoyle
Garry
Gambler
Gamsu
Gangnamjin
Gator
Geguri
Gesture
GiG
Gido
Glister
Gods
GodsB
Greyy
Guard
GuardiaN
Guxue
HOTBA
HaGoPeun
Haksal
Halo
Hanbin
Happy
HarryHook
Hawk
Heesu
Highly
Hooreg
HyP
Hydration
Hyeonu
ILLICIT
Izayaki
jiqiren
JJANU
JJonak
JAKE
Janus
Jaru
Jecse
Jerry
Jihun
JinMu
JMAC
KSAA
KSF
KSP
Kalios
Karayan
KariV
Kellex
Kodak
Krillin
Kris
Kruise
Krystal
KuKi
Kyb
Kyo


## Write to csv

In [43]:
df = pd.DataFrame(data=list_player)
df.to_csv("player_info.csv")
df = pd.DataFrame(data=history_list)
df.to_csv("player_history.csv")

## For debugging purposes